# Build AML Pipeline with custom module and built-in module

In this tutorial you will learn how to use Designer built-in module and custom module together to create a pipeline.

1. Setup enrivonment - install module CLI and module/pipeline SDK
2. Register custom modules into your aml workspace using CLI
3. Use module/pipeline SDK to create a pipeline with modules registered in step 2 and the built-in module available in AML designer

## Prerequisite
* Install Azure CLI, please follow [the Azure CLI installation instructions](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest) to install.

## Setup environment
* Install Azure CLI AML extension which includes the _module_ command group
* Install Azure ML SDK including the APIs to work with _module_ and _pipeline_

In [ ]:
# Uninstall azure-cli-ml (the `az ml` commands)
!az extension remove -n azure-cli-ml 

# Install local version of azure-cli-ml (which includes `az ml module` commands)
!az extension add --source https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/15510101/azure_cli_ml-0.1.0.15510101-py3-none-any.whl --pip-extra-index-urls https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/15510101 --yes

In [ ]:
# Verify the availability of `az ml module` commands
!az ml module -h

In [ ]:
# Install azureml-sdk with Pipeline, Module
# Important! After install succeed, need to restart kernel

%config IPCompleter.greedy=True 
!pip install azureml-pipeline-wrapper[notebooks]==0.1.0.15510101 --extra-index-url https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/15510101 --user --upgrade

In [ ]:
# you need to configure your ws information here

subscription_id = '74eccef0-4b8d-4f83-b5f9-fa100d155b22'
workspace_name = 'lisal-amlservice'
resource_group = 'lisal-dev'

# Specify available aml compute in workspace
pipeline_compute = "always-on-ds2v2"

## Register azureml module

You can manage AML module through [azure-cli-ml](https://aka.ms/moduledoc) or [ml.azure.com](https://ml.azure.com/). <br>

Module could be registered from:
- local path
- public Github url
- Azure DevOps build artifacts

In [ ]:
# Configure your aml workspace 

!az login 
!az account set -s $subscription_id 
!az ml folder attach -w $workspace_name -g $resource_group 

In [ ]:
# register a custom module

!az ml module register --spec-file=https://github.com/lisagreenview/hello-aml-modules/blob/master/train-score-eval/eval.yaml

In [ ]:
import json
from azureml.data.data_reference import DataReference
from azureml.core import Workspace, Run, Dataset, Datastore
from azureml.pipeline.wrapper import Pipeline, Module, dsl

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

In [ ]:
# get built-in module
select_column_func = Module.load(ws, namespace='azureml', name='Select Columns in Dataset')
clean_data_func = Module.load(ws, namespace='azureml', name='Clean Missing Data')
split_data_func = Module.load(ws, namespace='azureml', name='Split Data')
linear_regression_func = Module.load(ws, namespace='azureml', name='Linear Regression')
train_func = Module.load(ws, namespace='azureml', name='Train Model')
score_func = Module.load(ws, namespace='azureml', name='Score Model')
eval_func = Module.load(ws, namespace='azureml', name='Evaluate Model')

#get custom module
my_eval = Module.load(ws, namespace='microsoft.com/aml/samples', name='Evaluate')


data_name = 'Automobile_price_data'

if data_name not in ws.datasets:
    global_datastore = Datastore(ws, name="azureml_globaldatasets")
    automobile_data = Dataset.File.from_files(global_datastore.path('GenericCSV/Automobile_price_data_(Raw)')).register(workspace=ws, name='Automobile_price_data', description='Automobile_price_data')
    print('Registerd')

blob_input_data = ws.datasets[data_name]

In [ ]:
# define your pipeline

@dsl.pipeline(name = 'Designer Sample 1', 
              description = 'Regression - Automobile Price Prediction',
              default_compute_target = pipeline_compute)
def sample1_pipeline():
    select = select_column_func(
        dataset=blob_input_data, 
        select_columns='[{"ColumnNames":["normalized-losses"],"Exclude":true}]'
    )   
    
    clean = clean_data_func(
        dataset=select.outputs.results_dataset,
        columns_to_be_cleaned='{"isFilter":true,"rules":[{"ruleType":"AllColumns","exclude":false}]}',
        minimum_missing_value_ratio=0.0,
        maximum_missing_value_ratio=1.0,
        cleaning_mode='Remove entire row'
    )
    
    split = split_data_func(
        dataset=clean.outputs.cleaned_dataset,
        splitting_mode='Split Rows',
        fraction_of_rows_in_the_first_output_dataset=0.7,
        randomized_split='True',
        stratified_split='False'
    )
    
    algo = linear_regression_func(
        solution_method='Ordinary Least Squares',
        l2_regularization_weight=0.001,
        include_intercept_term='True',
        random_number_seed=0
    )
    
    train = train_func(
        dataset=split.outputs.results_dataset1,
        untrained_model=algo.outputs.untrained_model,
        label_column='[{"ColumnNames":["price"]}]'
    )
    
    score = score_func(
        trained_model=train.outputs.trained_model,
        dataset=split.outputs.results_dataset2,
        append_score_columns_to_output='True'
    )
    
    eval_1 = my_eval(
        scoring_result=score.outputs.scored_dataset
    )
       
    eval_2 = eval_func(
        scored_dataset = score.outputs.scored_dataset
    ) 
    
    return {**eval_1.outputs, **eval_2.outputs}

In [ ]:
# create a pipeline
pipeline = sample1_pipeline()

In [ ]:
# validate pipeline and visualize the graph
pipeline.validate()

In [ ]:
# save as a draft
pipeline.save(experiment_name = 'SDK-Created-Samples', id='ef7acf77-cd4b-4336-b4a2-c85536faa778')

In [ ]:
run = pipeline.submit(experiment_name = 'SDK-Created-Samples',tags={'mode':'SDK'})

In [ ]:
run.wait_for_completion()